<a href="https://colab.research.google.com/github/zzzle96/PyTorch/blob/master/convlstm%20basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvLSTM(nn.Module):
    def __init__(self, n_in, output_channel, class_num, in_ch):
        super(ConvLSTM, self).__init__()
        self.in_ch = in_ch
        self.n_in   = n_in
        self.out_ch = output_channel 
        self.rnn1 = Conv_LSTM(input_channel = self.in_ch, output_channel = self.out_ch)
        self.ap = nn.AvgPool1d(32,32, padding=0)
        self.fcl1 = nn.Linear(self.out_ch * self.n_in // 32, class_num, bias = False)
        
                
    def forward(self, x, hidden):
        h1 = hidden
        x = x.reshape(-1, self.in_ch, self.n_in)
        x, h1 = self.rnn1(x, h1)
        x = F.relu(x)
        x = self.ap(x)
        x = x.view(-1, self.out_ch * self.n_in // 32)
        x = self.fcl1(x)
        hidden = h1
                
        return x, hidden

class Conv_LSTM(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(Conv_LSTM, self).__init__()
        self.output_channel = output_channel
        self.fcl_wxf = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxi = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxo = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxg = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whf = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whi = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_who = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whg = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)

    def forward(self, input, hidden):
        h_t_1, c_t_1 = hidden
        f_t_i = (self.fcl_wxf(input) + self.fcl_whf(h_t_1))
        i_t_i = (self.fcl_wxi(input) + self.fcl_whi(h_t_1))
        g_t_i = (self.fcl_wxg(input) + self.fcl_whg(h_t_1))
        o_t_i = (self.fcl_wxo(input) + self.fcl_who(h_t_1))

        f_t = torch.sigmoid(f_t_i)
        i_t = torch.sigmoid(i_t_i)
        g_t = torch.tanh(g_t_i)
        o_t = torch.sigmoid(o_t_i)
                
        c_t = f_t * c_t_1 + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        output = h_t
        hidden = (h_t, c_t)
        
        return output, hidden
        
    def initHidden(self, batch_size, seg_size):
        weight = next(self.parameters())
        return (weight.new_zeros(batch_size, self.output_channel, seg_size),
                weight.new_zeros(batch_size, self.output_channel, seg_size))  